In [1]:
# Necessary Packages
!pip install tensorly-torch
!pip install tensorly


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

import tltorch # TCL and TRL
import tensorly as tl # Tensor operation
import torch   #  Neural Network
from torch import nn # Neural Network
from torch.autograd import Variable # Tensor input
import torch.optim as optim # Optimization
from torchvision import datasets, transforms # Datasets and transoforms
import torchvision # Data Loader
import torch.nn.functional as F # Activation Functions

import numpy as np # Numerical operations
import itertools # Generate indices

import matplotlib.pyplot as plt # Generate plots
import pandas as pd # Save the result as a sheet

import time # execuation time

import matplotlib.pyplot as plt # to plot and display the images

# from skimage import data # To load only one image

from reshape.reshape import MyReshape
from reshape.reshape import MyReshape2
from reshape.reshape import MyReshape3



from torch.utils.data import DataLoader, Subset, ConcatDataset # To add choose only a subset of dataset

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))])

batch_size = 4


trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

trainset_subset = Subset(trainset, range(0, 50))
trainloader = DataLoader(trainset_subset, batch_size=batch_size, shuffle=True, num_workers=2)

additional_trainset_subset = Subset(trainset, range(50, 60000))

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

combined_testset = ConcatDataset([testset, additional_trainset_subset])

testloader = DataLoader(combined_testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [5]:
# Get the shape of the data in training set
for images, labels in trainloader:
    print("Training set:")
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    break  # Print only the first batch

# Get the shape of the data in testing set
for images, labels in testloader:
    print("\nTesting set:")
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    break  # Print only the first batch

Training set:
Image batch shape: torch.Size([4, 1, 28, 28])
Label batch shape: torch.Size([4])

Testing set:
Image batch shape: torch.Size([4, 1, 28, 28])
Label batch shape: torch.Size([4])


In [6]:
class Net(nn.Module):
    def __init__(self, device):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.tcl = tltorch.TCL(input_shape=[2, 2, 2, 25, 2, 2], rank=[1, 1, 1, 13, 2, 2])
        self.trl = tltorch.TRL(input_shape=[13, 2, 2], output_shape=[10], factorization='Tucker', rank=[10, 3, 3, 10])
        self.device = device
        self.reshaper = MyReshape(device)

    def forward(self, x, map_type=1):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        
        x = self.reshaper(x, map_type)
        
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)

In [7]:
class Net2(nn.Module):
    def __init__(self, device):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.tcl = tltorch.TCL(input_shape=[2, 2, 2, 25, 2, 2], rank=[1, 1, 1, 13, 2, 2])
        self.trl = tltorch.TRL(input_shape=[13, 2, 2], output_shape=[10], factorization='Tucker', rank=[10, 3, 3, 10])
        self.device = device
        self.reshaper = MyReshape2(device)

    def forward(self, x, map_type=1):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        
        x = self.reshaper(x, map_type)
        
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)

In [8]:
class Net3(nn.Module):
    def __init__(self, device):
        super(Net3, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.tcl = tltorch.TCL(input_shape=[2, 2, 2, 25, 2, 2], rank=[1, 1, 1, 13, 2, 2])
        self.trl = tltorch.TRL(input_shape=[13, 2, 2], output_shape=[10], factorization='Tucker', rank=[10, 3, 3, 10])
        self.device = device
        self.reshaper = MyReshape3(device)

    def forward(self, x, map_type=1):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        
        x = self.reshaper(x, map_type)
        
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)

In [9]:
device = 'cuda'
n_epoch = 3
model = Net(device='cuda').to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

loss_1_Train = []
loss_1_Test = []

def train(epoch):
    model.train()
    total_loss = 0

    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data).to(device)
        loss = criterion(output, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss))
    
    avg_loss = total_loss / len(trainloader)
    loss_1_Train.append(avg_loss)





print("start train")

start_time = time.time()

for epoch in range(n_epoch):
    train(epoch)

end_time = time.time()

print(f"Training for all {n_epoch} epochs completed in: {end_time - start_time} seconds")

print("end train")


    

# def test():
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in testloader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += criterion(output, target).item()
#             pred = output.data.max(1, keepdim=True)[1]
#             correct += pred.eq(target.data.view_as(pred)).cpu().sum()

#     avg_test_loss = test_loss / len(testloader)
#     loss_1_Test.append(avg_test_loss)
#     print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         avg_test_loss, correct, len(testloader.dataset),
#         100. * correct / len(testloader.dataset)))




# st = time.time()
# for epoch in range(1, n_epoch + 1):
#     train(epoch)
#     test()

# elapsed_time = time.time() - st
# print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

start train
Train Epoch: 0 [0/50 (0%)]	Loss: 2.219562
Train Epoch: 0 [20/50 (38%)]	Loss: 2.272348
Train Epoch: 0 [40/50 (77%)]	Loss: 2.307836
Train Epoch: 1 [0/50 (0%)]	Loss: 2.161656
Train Epoch: 1 [20/50 (38%)]	Loss: 2.204363
Train Epoch: 1 [40/50 (77%)]	Loss: 2.191953
Train Epoch: 2 [0/50 (0%)]	Loss: 2.024172
Train Epoch: 2 [20/50 (38%)]	Loss: 2.269480
Train Epoch: 2 [40/50 (77%)]	Loss: 1.661605
Training for all 3 epochs completed in: 9.21672773361206 seconds
end train


In [10]:
device = 'cuda'
n_epoch = 3
model = Net2(device='cuda').to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

loss_1_Train = []
loss_1_Test = []

def train(epoch):
    model.train()
    total_loss = 0

    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data).to(device)
        loss = criterion(output, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss))
    
    avg_loss = total_loss / len(trainloader)
    loss_1_Train.append(avg_loss)

print("start train")
start_time = time.time()

for epoch in range(n_epoch):
    train(epoch)

end_time = time.time()

print(f"Training for all {n_epoch} epochs completed in: {end_time - start_time} seconds")

print("end train")


    

# def test():
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in testloader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += criterion(output, target).item()
#             pred = output.data.max(1, keepdim=True)[1]
#             correct += pred.eq(target.data.view_as(pred)).cpu().sum()

#     avg_test_loss = test_loss / len(testloader)
#     loss_1_Test.append(avg_test_loss)
#     print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         avg_test_loss, correct, len(testloader.dataset),
#         100. * correct / len(testloader.dataset)))

# st = time.time()
# for epoch in range(1, n_epoch + 1):
#     train(epoch)
#     test()

# elapsed_time = time.time() - st
# print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

start train
Train Epoch: 0 [0/50 (0%)]	Loss: 2.292078
Train Epoch: 0 [20/50 (38%)]	Loss: 2.319593
Train Epoch: 0 [40/50 (77%)]	Loss: 2.296949
Train Epoch: 1 [0/50 (0%)]	Loss: 2.264895
Train Epoch: 1 [20/50 (38%)]	Loss: 2.223021
Train Epoch: 1 [40/50 (77%)]	Loss: 2.359742
Train Epoch: 2 [0/50 (0%)]	Loss: 2.296491
Train Epoch: 2 [20/50 (38%)]	Loss: 2.269852
Train Epoch: 2 [40/50 (77%)]	Loss: 2.251085
Training for all 3 epochs completed in: 8.163865089416504 seconds
end train


In [11]:
device = 'cuda'
n_epoch = 3
model = Net3(device='cuda').to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

loss_1_Train = []
loss_1_Test = []

def train(epoch):
    model.train()
    total_loss = 0

    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data).to(device)
        loss = criterion(output, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % 5 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss))
    
    avg_loss = total_loss / len(trainloader)
    loss_1_Train.append(avg_loss)

print("start train")
start_time = time.time()

for epoch in range(n_epoch):
    train(epoch)

end_time = time.time()

print(f"Training for all {n_epoch} epochs completed in: {end_time - start_time} seconds")

print("end train")


start train


ValueError: shapes torch.Size([4, 4, 2, 25, 2, 2]) and torch.Size([1, 2]) not aligned in mode-1 multiplication: 4 (mode 1) != 2 (dim 1 of matrix)

In [20]:
import torch
t = torch.tensor([[[1,2],[3,4]],[[5,6],[7,8]]])

from reshape.reshape import MyReshape
from reshape.reshape import MyReshape2


r1 = MyReshape('cpu')
r2 = MyReshape2('cpu')

t1 = r1(t,map_type=1)
# t2 = r2(t)

RuntimeError: INDICES element is out of DATA bounds, id=2 axis_dim=2